## 국가보안기술연구소(NSRI)
- 함수의 시작 구분 프로젝트 by ISYS Lab

## Input 
- objdump -d 로 바이트를 뽑아서 저장한 .txt 파일 (16진수 hex 코드 & 함수이름 섞여있음)

## output
- 해당 바이트코드와 함수의 시작여부(bin) 형태의 csv 파일

In [2]:
# preprocessing 함수 (0, 1) 이중분류
# 16진수 가져와서 함수가 적혀있는 행은 버리고 , 그 다음항목 1로 마킹
# 500000만번 마다 체크
def preprocessing(k):
    p = pd.DataFrame(np.array(k.split()))
    p['label'] = 0 
    p.columns = ['bin', 'label']
    
    f_index = [] # 함수 위치 index 담은 list 
    f_name = [] # 함수 이름 담은 list
    for idx in range(0, len(p)):
        # 진행현황 체크
        if idx % 500000 == 0:
            print(idx,'Finish', end=' ') # 진행속도 체크 함수

        if len(str(p['bin'][idx])) !=2: # 칸의 이름이 2글자가 아니면
            f_index.append(idx); f_name.append(str(p['bin'][idx])) # 함수 이름 저장 list
            p['label'][idx+1] = 1 # bin 항목 1로 마킹
            #print(idx, end=' ')#, p['num'][idx])
            
    # 함수이름 행들 버리기
    drop_index = p['bin'][f_index].index # drop 할 index
    p = p.drop(drop_index, 0) # drop 행 버리기
    print('전체 행 갯수:', len(p), '함수갯수:', len(drop_index),'byte갯수:', len(p)-len(drop_index))
    
    return f_index, f_name, p # 함수 위치 index list , 함수 이름 list, 최종 df 

In [7]:
######################### main 함수
# gcc v_start ~ v_end - optimization 0~4
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

#v_start, v_end = 6, 7 # version 정보 - ex 6,10 = gcc 6버전부터 ~ 9버전까지
v_start, v_end = 7, 10
o_start, o_end = 0, 4 # optimizaton 정보 - ex 0, 4 = 최적화 O0, O1, O2, O3 까지

dir_path = 'data/coretils_gcc3~9_op0~4_exec_txt/'
#dir_path = 'data/binutils_gcc3~9_op0~4_txt/'

bit = '32'

f_index_ls = [] # 함수 index ls
f_name_ls = [] # 함수 이름 ls
df_ls = [] # 데이터프레임 ls

# gcc version 별로
for version in range(v_start, v_end):
    for optimization in range(o_start, o_end):    
        # 파일 경로 , gcc 버전, op 버전 지정
        start_time = time.time() # 소요시간 측정용
        file_name = dir_path +'core_gcc' + str(version) + '_' + str(optimization) + '_' + bit + '.txt'
        
        # 파일 open
        print(file_name, '시작')
        f = open(file_name, 'r')
        
        # 파일 읽어들이기 완료, 전처리 시작
        index_ls, name_ls, p_ls = preprocessing(f.read())
        f_index_ls.append(index_ls); f_name_ls.append(name_ls); df_ls.append(p_ls)
        
        # 파일 전처리 종료
        end_time = time.time() # 소요시간 측정용
        print(file_name, '완료')
        print('걸린시간:', np.round(end_time-start_time,1),'초\n\n')

data/coretils_gcc3~9_op0~4_exec_txt/core_gcc7_0_32.txt 시작
0 Finish 500000 Finish 1000000 Finish 1500000 Finish 2000000 Finish 2500000 Finish 3000000 Finish 3500000 Finish 4000000 Finish 전체 행 갯수: 4288347 함수갯수: 24555 byte갯수: 4263792
data/coretils_gcc3~9_op0~4_exec_txt/core_gcc7_0_32.txt 완료
걸린시간: 167.5 초


data/coretils_gcc3~9_op0~4_exec_txt/core_gcc7_1_32.txt 시작
0 Finish 500000 Finish 1000000 Finish 1500000 Finish 2000000 Finish 2500000 Finish 3000000 Finish 전체 행 갯수: 3403824 함수갯수: 19477 byte갯수: 3384347
data/coretils_gcc3~9_op0~4_exec_txt/core_gcc7_1_32.txt 완료
걸린시간: 121.8 초


data/coretils_gcc3~9_op0~4_exec_txt/core_gcc7_2_32.txt 시작
0 Finish 500000 Finish 1000000 Finish 1500000 Finish 2000000 Finish 2500000 Finish 3000000 Finish 전체 행 갯수: 3413150 함수갯수: 19251 byte갯수: 3393899
data/coretils_gcc3~9_op0~4_exec_txt/core_gcc7_2_32.txt 완료
걸린시간: 122.2 초


data/coretils_gcc3~9_op0~4_exec_txt/core_gcc7_3_32.txt 시작
0 Finish 500000 Finish 1000000 Finish 1500000 Finish 2000000 Finish 2500000 Finish 3000

## 파일저장 

In [8]:
####################### 각 gcc-version-optimization 별 한번에 저장 

for idx, d in enumerate(df_ls):    
    # 한번에 저장
    d.to_csv('gcc'+str(v_start)+'_'+str(idx)+'_32_exec_coreutils.csv')
    print('gcc'+str(v_start)+'_'+str(idx)+'_32_exec_coreutils.csv 생성완료')

gcc7_0_32_exec_coreutils.csv 생성완료
gcc7_1_32_exec_coreutils.csv 생성완료
gcc7_2_32_exec_coreutils.csv 생성완료
gcc7_3_32_exec_coreutils.csv 생성완료
gcc7_4_32_exec_coreutils.csv 생성완료
gcc7_5_32_exec_coreutils.csv 생성완료
gcc7_6_32_exec_coreutils.csv 생성완료
gcc7_7_32_exec_coreutils.csv 생성완료
gcc7_8_32_exec_coreutils.csv 생성완료
gcc7_9_32_exec_coreutils.csv 생성완료
gcc7_10_32_exec_coreutils.csv 생성완료
gcc7_11_32_exec_coreutils.csv 생성완료


In [5]:
# test read
gcc6_0_32exec = pd.read_csv('gcc6_0_32_exec.csv')
print(gcc6_0_32exec.shape)
gcc6_0_32exec.head()


(13936744, 3)


,Unnamed: 0,bin,label
0,1,53,1
1,2,83,0
2,3,ec,0
3,4,08,0
4,5,e8,0


In [6]:
gcc6_0_32exec['label'].value_counts()

0    13892354
1       44390
Name: label, dtype: int64

In [5]:
########## 각 gcc-version-optimization 별 저장 -> gcc7 , 0123, 32 , 012 다중분류

gcc_v7_o = ['gcc7_0_32','gcc7_1_32','gcc7_2_32','gcc7_3_32']

for idx, d in enumerate(df_ls):    
    # 한번에 저장
    d.to_csv(gcc_v7_o[idx]+'_012.csv')
    print(gcc_v7_o[idx], '생성완료')

gcc7_0_32 생성완료
gcc7_1_32 생성완료
gcc7_2_32 생성완료
gcc7_3_32 생성완료


In [9]:
# test read
x = pd.read_csv('gcc7_0_32_012.csv')
print(x.shape)
x[label].value_counts()

(2685709, 3)


0    2685708
1          1
Name: bin, dtype: int64

In [ ]:
########## 각 gcc-version-optimization 별 저장 -> gcc6 , 0123, 32 , 012 다중분류

gcc_v6_o = ['gcc6_0_32','gcc6_1_32','gcc6_2_32','gcc6_3_32']

for idx, d in enumerate(df_ls):    
    # 한번에 저장
    d.to_csv(gcc_v6_o[idx]+'_012.csv')
    print(gcc_v6_o[idx], '생성완료')

## 파일 3등분 저장

In [67]:
####################### 각 gcc-version-optimization 별  3등분 분리 
for idx, d in enumerate(df):
    df_len = int(len(d) / 3) # 3등분 길이
    a, b ,c = d.loc[0:df_len], d.loc[df_len:df_len*2], d.loc[df_len*2:df_len*3]
    
    # gcc 버전별 3등분된 길이 a, b, c 로 나눔
    print(gcc_v_o[idx], ':', len(a), len(b), len(c))
    a.to_csv('binutils_gcc3~9_op0~4_divide-3_csv/' + gcc_v_o[idx]+'_a.csv')
    b.to_csv('binutils_gcc3~9_op0~4_divide-3_csv/' + gcc_v_o[idx]+'_b.csv')
    c.to_csv('binutils_gcc3~9_op0~4_divide-3_csv/' + gcc_v_o[idx]+'_c.csv')

    print(gcc_v_o[idx], 'a,b,c 생성완료')

gcc6_0_32 : 776033 775924 773068
gcc6_0_32 a,b,c 생성완료
gcc6_1_32 : 593205 593064 590187
gcc6_1_32 a,b,c 생성완료
gcc6_2_32 : 628288 628115 625241
gcc6_2_32 a,b,c 생성완료
gcc6_3_32 : 719877 719746 716737
gcc6_3_32 a,b,c 생성완료
gcc7_0_32 : 887929 887927 887966
gcc7_0_32 a,b,c 생성완료
gcc7_1_32 : 623923 623884 623963
gcc7_1_32 a,b,c 생성완료
gcc7_2_32 : 643566 643513 643574
gcc7_2_32 a,b,c 생성완료
gcc7_3_32 : 747989 747937 747948
gcc7_3_32 a,b,c 생성완료
gcc8_0_32 : 776033 775924 773068
gcc8_0_32 a,b,c 생성완료
gcc8_1_32 : 593205 593064 590187
gcc8_1_32 a,b,c 생성완료
gcc8_2_32 : 628288 628115 625241
gcc8_2_32 a,b,c 생성완료
gcc8_3_32 : 719877 719746 716737
gcc8_3_32 a,b,c 생성완료
gcc9_0_32 : 887929 887927 887966
gcc9_0_32 a,b,c 생성완료
gcc9_1_32 : 623923 623884 623963
gcc9_1_32 a,b,c 생성완료
gcc9_2_32 : 643566 643513 643574
gcc9_2_32 a,b,c 생성완료
gcc9_3_32 : 747989 747937 747948
gcc9_3_32 a,b,c 생성완료
